In [1]:
import pandas as pd
from scipy import stats
from statistics import mean
from datetime import datetime as DateTime
import yfinance as yf
import pytickersymbols as pts
import stocks_utils as sou

## Acquiring information about stocks 

In [2]:
stock_data = pts.PyTickerSymbols()

us30_stocks = stock_data.get_stocks_by_index('DOW JONES')
nasdaq_stocks = stock_data.get_stocks_by_index('NASDAQ 100')
us500_stocks = stock_data.get_stocks_by_index('S&P 500')
uk100_stocks = stock_data.get_stocks_by_index('FTSE 100')
eu50_stocks = stock_data.get_stocks_by_index('EURO STOXX 50')

market_stocks = list(us30_stocks) + list(nasdaq_stocks) + list(us500_stocks) + list(uk100_stocks) + list(eu50_stocks)

excluded_industries = ['Online gambling','Healthcare','Managed Health care','Banking & Investment Services','Metals & Mining','Metal','Insurance','Commercial REITs','Specialized REITs','Financial services','Real Estate','Casinos & Gaming','Fossil Fuels','Defense','Cosmetics','defense','Life insurance','Real estate investment trust']

# excluded_sectors = ['Biotechnology & Medical Research','Online gambling','Pharmaceuticals','Healthcare','Managed Health care','Medical Equipment','Banking & Investment Services','Medical Equipment, Supplies & Distribution','Metals & Mining','Metal','Insurance','Commercial REITs','Specialized REITs','Financial services','Beverages','Food & Beverages','Real Estate','Casinos & Gaming','Fossil Fuels','Defense','Restaurants']

stocks_list = []
for market_stock in market_stocks:
    try:
        ignore = False
        for market_industry in market_stock['industries']:
            if(market_industry in excluded_industries):
                ignore = True
                break
            
        if(ignore):
            continue

        stock_symbol = market_stock['symbol']
        symbols = market_stock['symbols']

        symbol_google = None
        symbol_yahoo = None
        for symbol in symbols:
            if(symbol['google'].endswith(f':{stock_symbol}')):
                symbol_google = symbol['google']
                symbol_yahoo = symbol['yahoo']
                break

        if(symbol_yahoo is None):
            continue

        item = {
            'Company': market_stock['name'],
            'Symbol': stock_symbol,
            'Symbol_Google': symbol_google,
            'Symbol_Yahoo': symbol_yahoo,
            'Country': market_stock['country'],
            'Indices': ', '.join(market_stock['indices']),
            'Industries': ', '.join(market_stock['industries'])
        }

        stocks_list.append(item)
    except:
        pass

stocks = pd.DataFrame(stocks_list, columns=['Company','Symbol','Symbol_Google','Symbol_Yahoo','Country','Indices','Industries'])
stocks

,Company,Symbol,Symbol_Google,Symbol_Yahoo,Country,Indices,Industries
0,3M Company,MMM,FRA:MMM,MMM.F,United States,"DOW JONES, S&P 100, S&P 500","Industrials, Industrial Conglomerates"
1,Apple Inc.,AAPL,NASDAQ:AAPL,AAPL,United States,"DOW JONES, NASDAQ 100, S&P 100, S&P 500","Computers, Phones & Household Electronics, Tec..."
2,The Boeing Company,BA,NYSE:BA,BA,United States,"DOW JONES, S&P 100, S&P 500","Industrial Goods, Industrials, Aerospace & Def..."
3,Caterpillar Inc.,CAT,NYSE:CAT,CAT,United States,"DOW JONES, S&P 100, S&P 500","Industrial Goods, Industrials, Heavy Machinery..."
4,Cisco Systems Inc.,CSCO,NASDAQ:CSCO,CSCO,United States,"DOW JONES, NASDAQ 100, S&P 100, S&P 500","Technology, Technology Equipment, Communicatio..."
...,...,...,...,...,...,...,...
482,Amadeus IT Group SA,AMS,BME:AMS,AMS.MC,Spain,"EURO STOXX 50, IBEX 35","Technology, Software & IT Services, IT Service..."
483,Iberdrola SA,IBE,BME:IBE,IBE.MC,Spain,"EURO STOXX 50, IBEX 35","Electric Utilities, Electric Utilities & IPPs,..."
484,Koninklijke Ahold Delhaize N.V.,AD,AMS:AD,AD.AS,Netherlands,"AEX, EURO STOXX 50","Consumer Non-Cyclicals, Food & Drug Retailing,..."
485,ASML Holding N.V.,ASML,NASDAQ:ASML,ASML,Netherlands,"AEX, EURO STOXX 50, NASDAQ 100","Semiconductor Equipment & Testing, Technology,..."


In [3]:
# Some stocks are listed on more than the stock index.
stocks.set_index('Symbol',inplace=True)
stocks = stocks[~stocks.index.duplicated(keep='first')]
stocks

,Company,Symbol_Google,Symbol_Yahoo,Country,Indices,Industries
Symbol,,,,,,
MMM,3M Company,FRA:MMM,MMM.F,United States,"DOW JONES, S&P 100, S&P 500","Industrials, Industrial Conglomerates"
AAPL,Apple Inc.,NASDAQ:AAPL,AAPL,United States,"DOW JONES, NASDAQ 100, S&P 100, S&P 500","Computers, Phones & Household Electronics, Tec..."
BA,The Boeing Company,NYSE:BA,BA,United States,"DOW JONES, S&P 100, S&P 500","Industrial Goods, Industrials, Aerospace & Def..."
CAT,Caterpillar Inc.,NYSE:CAT,CAT,United States,"DOW JONES, S&P 100, S&P 500","Industrial Goods, Industrials, Heavy Machinery..."
CSCO,Cisco Systems Inc.,NASDAQ:CSCO,CSCO,United States,"DOW JONES, NASDAQ 100, S&P 100, S&P 500","Technology, Technology Equipment, Communicatio..."
...,...,...,...,...,...,...
AIR,Airbus SE,FRA:AIR,AIR.F,Netherlands,"MDAX, EURO STOXX 50, CAC 40","Industrial Goods, Industrials, Aerospace & Def..."
AMS,Amadeus IT Group SA,BME:AMS,AMS.MC,Spain,"EURO STOXX 50, IBEX 35","Technology, Software & IT Services, IT Service..."
IBE,Iberdrola SA,BME:IBE,IBE.MC,Spain,"EURO STOXX 50, IBEX 35","Electric Utilities, Electric Utilities & IPPs,..."


## Transforming data for value calculation 

In [4]:
# Defining stocks value scoring dataframe

df_columns = [
    'Company',
    'Symbol',
    'Symbol_Google',
    'Symbol_Yahoo',
    'Description',
    'Country',        
    'Indices',
    'Industries',
    'MarketCap',
    'Y1HighestPriceValue',
    'Y1HighestPriceDate',
    'Currency',
    'PriceToEarningsRatio',
    'PriceToEarningsPercentile',
    'PriceToEarningsInversePercentile',
    'PriceToSalesRatio',
    'PriceToSalesPercentile',
    'PriceToSalesInversePercentile',
    'EV_EBITDA',
    'EV_EBITDAPercentile',
    'EV_R',
    'EV_RPercentile',
    'EV_RInversePercentile',
    'Score_Long',
    'Score_Short'
]

so_dataframe = pd.DataFrame(columns = df_columns)

# Getting fundamental data
i = 0
total_stocks = len(stocks)
errors = 0
for index, stock in stocks.iterrows():
    i+=1
    stock_name = stock['Company']
    symbol = index

    symbol_google = stock['Symbol_Google']
    symbol_yahoo = stock['Symbol_Yahoo']

    print(f'({i}/{total_stocks}) | {symbol} (Google: {symbol_google}, Yahoo: {symbol_yahoo}) | {stock_name}')
    print(f'Retreiving fundamental data from yfinance...')

    try:
        yf_ticker = yf.Ticker(symbol_yahoo)

        day_data = yf_ticker.history(period="1y", interval="1d")
        day_data.dropna(inplace=True)
        day_data.sort_values('High', ascending=False, inplace=True)

        so_dataframe = so_dataframe.append(
            pd.Series([
                stock_name,
                symbol,
                symbol_google,
                symbol_yahoo,
                yf_ticker.info['longBusinessSummary'],
                stock['Country'],
                stock['Indices'],
                stock['Industries'],
                yf_ticker.info['marketCap'],
                day_data.iloc[0]['High'],
                day_data.index[0].strftime('%Y-%m-%d'),
                yf_ticker.info['currency'],
                yf_ticker.info['trailingPE'],
                'N/A',
                'N/A',
                yf_ticker.info['priceToSalesTrailing12Months'],
                'N/A',
                'N/A',
                yf_ticker.info['enterpriseToEbitda'],
                'N/A',
                yf_ticker.info['enterpriseToRevenue'],
                'N/A',
                'N/A',
                'N/A',
                'N/A'
            ], index = df_columns),
            ignore_index = True)

    except Exception as error:
        print('ERROR:', error)
        errors+=1
        print('An error occurred while trying to get the information!')
        
    print()

initial_count = len(so_dataframe)

# Dealing With Missing Data
so_dataframe = so_dataframe[so_dataframe['Industries']!=''].dropna()

(1/394) | MMM (Google: FRA:MMM, Yahoo: MMM.F) | 3M Company
Retreiving fundamental data from yfinance...

(2/394) | AAPL (Google: NASDAQ:AAPL, Yahoo: AAPL) | Apple Inc.
Retreiving fundamental data from yfinance...

(3/394) | BA (Google: NYSE:BA, Yahoo: BA) | The Boeing Company
Retreiving fundamental data from yfinance...
ERROR: 'trailingPE'
An error occurred while trying to get the information!

(4/394) | CAT (Google: NYSE:CAT, Yahoo: CAT) | Caterpillar Inc.
Retreiving fundamental data from yfinance...

(5/394) | CSCO (Google: NASDAQ:CSCO, Yahoo: CSCO) | Cisco Systems Inc.
Retreiving fundamental data from yfinance...

(6/394) | KO (Google: OTCMKTS:KO, Yahoo: KO) | The Coca-Cola Company
Retreiving fundamental data from yfinance...

(7/394) | HD (Google: NYSE:HD, Yahoo: HD) | The Home Depot Inc.
Retreiving fundamental data from yfinance...

(8/394) | IBM (Google: FRA:IBM, Yahoo: IBM.F) | International Business Machines Corporation
Retreiving fundamental data from yfinance...

(9/394) | IN

In [17]:
# Calculating percentiles
print('Calculating percentiles...')

metrics_long = {
    'PriceToEarningsRatio': 'PriceToEarningsPercentile',
    'PriceToSalesRatio': 'PriceToSalesInversePercentile',
    'EV_EBITDA':'EV_EBITDAPercentile',
    'EV_R':'EV_RInversePercentile'
}

metrics_short = {
    'PriceToEarningsRatio': 'PriceToEarningsInversePercentile',
    'PriceToSalesRatio': 'PriceToSalesPercentile',
    'EV_R':'EV_RPercentile'
}

for row in so_dataframe.index:
    for metric in metrics_long.keys():
        if(metric == 'PriceToSalesRatio' or metric == 'EV_R'):
            so_dataframe.loc[row, metrics_long[metric]] = stats.percentileofscore(1 / so_dataframe[metric], 1 / so_dataframe.loc[row, metric])
        else:
            so_dataframe.loc[row, metrics_long[metric]] = stats.percentileofscore(so_dataframe[metric], so_dataframe.loc[row, metric])

    for metric in metrics_short.keys():
        if(metric == 'PriceToEarningsRatio'):
            so_dataframe.loc[row, metrics_short[metric]] = stats.percentileofscore(1 / so_dataframe[metric], 1 / so_dataframe.loc[row, metric])
        else:
            so_dataframe.loc[row, metrics_short[metric]] = stats.percentileofscore(so_dataframe[metric], so_dataframe.loc[row, metric])
            

# Scoring stocks
print('Scoring stocks...')
for row in so_dataframe.index:
    value_percentiles = []
    for metric in metrics_long.keys():
        value_percentiles.append(so_dataframe.loc[row, metrics_long[metric]])
    so_dataframe.loc[row, 'Score_Long'] = mean(value_percentiles)

    value_percentiles = []
    for metric in metrics_short.keys():
        value_percentiles.append(so_dataframe.loc[row, metrics_short[metric]])
    so_dataframe.loc[row, 'Score_Short'] = mean(value_percentiles)

# Sorting and selecting
result_df = so_dataframe.copy()
result_df.sort_values(by = 'Company', inplace = True, ascending=True)
result_df['MarketCap_USD'] = result_df['MarketCap'] * result_df['Currency'].apply(sou.get_usd_rate)
result_df = result_df[['Company','Symbol','Symbol_Google','Symbol_Yahoo','Description','Country','Indices','Industries','Currency','MarketCap','MarketCap_USD','Y1HighestPriceValue','Y1HighestPriceDate', 'PriceToEarningsRatio','PriceToEarningsPercentile','PriceToEarningsInversePercentile','PriceToSalesRatio','PriceToSalesPercentile','PriceToSalesInversePercentile','EV_EBITDA','EV_EBITDAPercentile','EV_R','EV_RPercentile','EV_RInversePercentile','Score_Long','Score_Short']]
skipped_count = initial_count - len(result_df)
now = DateTime.now()

print()
print(f'Scoring executed successfully at {now.hour}:{now.minute} | {total_stocks} listed | {errors} not found | {skipped_count} skipped | {len(result_df)} scored stocks')

with open("md.txt", "a", encoding='utf-8') as f:
    md_df = result_df[['Company','Symbol','Symbol_Google','Symbol_Yahoo','Currency','MarketCap','MarketCap_USD','PriceToEarningsRatio','PriceToEarningsPercentile','PriceToEarningsInversePercentile','PriceToSalesRatio','PriceToSalesPercentile','PriceToSalesInversePercentile','EV_EBITDA','EV_EBITDAPercentile','EV_R','EV_RPercentile','EV_RInversePercentile','Score_Long','Score_Short']]
    md_df.columns = ['Company','Symbol','Google Symbol','Yahoo Symbol', 'Currency','Market Cap', 'Market Cap (USD)','P/E', 'P/E PC', 'P/E INV-PC','P/S','P/S PC','P/S INV-PC','EV/EBITDA','EV/EBITDA PC','EV/R','EV/R PC','EV/R INV-PC','Score for Long','Score for Short']
    f.write(md_df.to_markdown(index=False))

Calculating percentiles...
Scoring stocks...

Scoring executed successfully at 19:24 | 394 listed | 75 not found | 14 skipped | 305 scored stocks


In [18]:
writer = pd.ExcelWriter('market_stock_value_scoring.xlsx', engine='xlsxwriter')
result_df.to_excel(writer, sheet_name='Value Scoring', index = False)

# Defining Excel columns formats and templates
background_light_color = '#ffffff'
background_dark_color = '#eeeeee'
background_accent_color = '#0086CD'
black_color = '#000000'
white_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': black_color,
            'bg_color': background_light_color,
            'border': 0
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': black_color,
            'bg_color': background_light_color,
            'border': 0
        }
    )

percentile_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': black_color,
            'bg_color': background_dark_color,
            'border': 0
        }
    )

score_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': white_color,
            'bg_color': background_accent_color,
            'border': 0
        }
    )

print('Saving scoring in Excel...\n')

column_formats = {
    'A': ['Company', string_template],
    'B': ['Symbol', string_template],
    'C': ['Symbol_Google', string_template],
    'D': ['Symbol_Yahoo', string_template],
    'E': ['Description', string_template],
    'F': ['Country', string_template],
    'G': ['Indices', string_template],
    'H': ['Industries', string_template],
    'I': ['Currency', string_template],
    'J': ['MarketCap', float_template],
    'K': ['MarketCap_USD', float_template],
    'L': ['Y1HighestPriceValue', float_template],
    'M': ['Y1HighestPriceDate', string_template],
    'N': ['PriceToEarningsRatio', float_template],
    'O': ['PriceToEarningsPercentile', percentile_template],
    'P': ['PriceToEarningsInversePercentile', percentile_template],
    'Q': ['PriceToSalesRatio', float_template],
    'R': ['PriceToSalesPercentile', percentile_template],
    'S': ['PriceToSalesInversePercentile', percentile_template],
    'T': ['EV_EBITDA', float_template],
    'U': ['EV_EBITDAPercentile', percentile_template],
    'V': ['EV_R', float_template],
    'W': ['EV_RPercentile', percentile_template],
    'X': ['EV_RInversePercentile', percentile_template],
    'Y': ['Score_Long', score_template],
    'Z': ['Score_Short', score_template]
}

for column in column_formats.keys():
    writer.sheets['Value Scoring'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Scoring'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.save()

Saving scoring in Excel...

